In [1]:
import yaml
import os
import sys
import csv
import pystache
import pickle
import re
from pyeda.inter import *
import random
import shutil

In [2]:
#!pip install yaml
#!pip install pystache

In [3]:
# pandoc assignment.md -o assignment.html --mathml -s --metadata title=" "
# pandoc assignment.md -o assignment.pdf -V geometry:margin=1in 

In [4]:
DEFAULT_INFILE = "/Users/mbs/Google Drive/Courses/ECEM16/2020 Spring/psets/pset1/main.yaml"
DEFAULT_DESTDIR = "/Users/mbs/Courses/ECEM16/Spring 2020/psets/pset1"
class AssignmentGenerator:
    def __init__(self, assignment=DEFAULT_INFILE, dirname_dst=DEFAULT_DESTDIR, params={}):
        self.assignment = os.path.normpath(assignment)
        self.dirname_dst = os.path.normpath(dirname_dst)
        self.basename = os.path.basename(self.assignment)
        self.dirname_ass = os.path.dirname(self.assignment)
        self.resetData()
        
    def resetData(self):
        if not os.path.exists(self.dirname_dst):
            os.makedirs(self.dirname_dst)
        os.chdir(self.dirname_dst)
        self.output_dirname = os.path.join(self.dirname_dst,"outputs") 
        if os.path.exists(self.output_dirname):
            shutil.rmtree(self.output_dirname)
        os.makedirs(self.output_dirname)
        self.instructions = None
        self.data = None
        self.roster = None
        self.info_for_grading = {}
        
    def loadData(self):
        self.openSpecificationFile()
        self.processSpecification()
        self.readProblemFiles()
        self.readInstructionsFile()
                
    @staticmethod
    def render(source_content, args):
        def replexpr(match):
            return str(eval(match.group(1)))
        source_content = str(source_content)
        try:
            content_1 = pystache.render(source_content, args)
        except:
            #import pdb; pdb.set_trace()
            raise SystemExit("Error in Mustache template:\n{0}\n with parameters\n{1}\n".format(source_content,args))
        p = re.compile(r'{%(.*?)%}')
        return p.sub(replexpr, content_1)

In [5]:
def openSpecificationFile(self):
    try:
        with open(self.assignment) as f:
            self.data = yaml.load(f, Loader=yaml.FullLoader)
    except IOError as e:
        print(e)
    except yaml.scanner.ScannerError as e:
        print("Input file {0} has error: {1}".format(self.basename,e))
    except:
        print("Input file {0} has error: unknown".format(self.basename))

setattr(AssignmentGenerator, 'openSpecificationFile', openSpecificationFile)

In [6]:
def processSpecification(self):
    try:
        self.problems = self.data['problems']
    except:
        print("Error: Empty assignment as no problems are specified in {0}".format(self.basename))

    if not 'problem_bank' in self.data:
        self.problem_bank = os.path.abspath(os.path.join(self.dirname_dst,"problem_bank"))
        print("Warning: No problem_bank specified in {0}. Using {1}".format(self.basename,self.problem_bank))
    else:
        self.problem_bank_dirname = os.path.abspath(os.path.join(self.dirname_ass,self.data['problem_bank']))

    if not os.path.exists(self.problem_bank_dirname):
        print("Error: problem bank directory {0} does not exist.".format(self.problem_bank_dirname))

setattr(AssignmentGenerator, 'processSpecification', processSpecification)

In [7]:
def readProblemFiles(self):
    probnum = 0
    for p in self.problems:
        probnum += 1
        if not p:
            raise SystemExit("Problem # {0} is not properly specified.".format(probnum))
        if not ('file' in p or 'subproblems' in p or 'randompicks' in p):
            raise SystemExit("Problem # {0} has no file or subproblems or randompicks.".format(probnum))
        if 'file' in p:
            try:
                problem_filename = os.path.join(self.problem_bank_dirname,p['file'])
                with open(problem_filename) as f:
                    p['template'] = (f.read()).strip().rstrip()
            except IOError as e:
                SystemExit(e)
            except:
                SystemExit("Error: unable to read {0}".format(problem_filename))
            p['directory'] = os.path.dirname(problem_filename)
        elif 'randompicks' in p:
            randompicknum = 0
            for sp in p['randompicks']:
                randompicknum += 1
                if not sp:
                    SystemExit("Problem # {0} [option {1}] is not properly specified.".format(probnum,randompicknum))
                if not 'file' in sp:
                    SystemExit("Problem # {0} [option {1}] has no file specified.".format(probnum,randompicknum))
                try:
                    randompick_filename = os.path.join(self.problem_bank_dirname,sp['file'])
                    with open(randompick_filename) as f:
                        sp['template'] = (f.read()).strip().rstrip()
                except IOError as e:
                    SystemExit(e)
                except:
                    SystemExit("Error: unable to read {0}".format(randompick_filename))
                sp['directory'] = os.path.dirname(randompick_filename)
        else:
            subprobnum = 0
            for sp in p['subproblems']:
                if not sp:
                    SystemExit("Problem # {0} [option {1}] is not properly specified.".format(probnum,randompicknum))
                if not 'file' in sp:
                    SystemExit("Error: Problem # {0}_{1} has no file specified.".format(probnum,subprobnum))
                subprobnum += 1
                try:
                    subproblem_filename = os.path.join(self.problem_bank_dirname,sp['file'])
                    with open(subproblem_filename) as f:
                        sp['template'] = (f.read()).strip().rstrip()
                except IOError as e:
                    SystemExit(e)
                except:
                    SystemExit("Error: unable to read {0}".format(subproblem_filename))
                sp['directory'] = os.path.dirname(subproblem_filename)
                
setattr(AssignmentGenerator, 'readProblemFiles', readProblemFiles)

In [8]:
def readInstructionsFile(self):
    if not 'instructions' in self.data:
        self.data['instructions'] = "./instructions.md"
    instruction_filename = os.path.abspath(os.path.join(self.dirname_ass,self.data['instructions']))
    try: 
        with open(instruction_filename) as f:
            self.instructions = f.read()
    except:
        SystemExit("Error: unable to read instruction file {0}".format(instruction_filename))
        
setattr(AssignmentGenerator, 'readInstructionsFile', readInstructionsFile)

In [9]:
def readRosterFile(self,roster_filename=None):
    if not roster_filename and 'roster' in self.data:
        roster_filename = os.path.abspath(os.path.join(self.dirname_ass,self.data['roster']))
        
    if not roster_filename:
        self.roster = [["", "", ""]]
    else:
        with open(roster_filename, newline='') as csvfile:
            roster_reader = csv.reader(csvfile, delimiter=',', 
                                       quotechar='|', skipinitialspace=True)
            self.roster = []
            for row in roster_reader:
                self.roster.append([row[0].strip(), row[1].strip().split()[0],row[2].strip().split()[0]])

setattr(AssignmentGenerator, 'readRosterFile', readRosterFile)

In [10]:
def processOneProblem(self,problem,args,dirname):
    all_content = problem['template'].strip().rstrip()
    yaml_header = {}
    if all_content[:3]=="---":
        i = all_content[4:].find("---")
        if i!=-1:
            try:
                yaml_header = yaml.full_load(all_content[0:i+4])
                all_content = all_content[i+8:].strip()
            except yaml.scanner.ScannerError as e:
                raise SystemExit("Header of problem {0} has error: {1}".format(problem['file'],e))
            except:
                raise SystemExit("Header of problem {0} has error".format(problem['file']))
    #import pdb; pdb.set_trace()
    #assert False
    args.update(yaml_header.get('defaults',{}))
    args.update(problem.get('params',{}))
    all_content = self.render(all_content, args)
    answer_key = yaml_header.get('answer_key',[])
    assets_required = yaml_header.get('assets',[])
    for ak in answer_key:
        ak['answer'] = self.render(ak.get('answer',""), args)
    answer_key = answer_key
    if all_content[-3:]=="```":
        i = all_content[:-3].rfind("```")
        if i!=-1:
            problem_content = all_content[:i]
            answer_sheet = all_content[i:-3]
            i =  answer_sheet.find("#")
            if i!=-1:
                answer_sheet = answer_sheet[i:]
            else:
                answer_sheet = ""
        else:
            problem_content = all_content
            answer_sheet = ""
    else:
        problem_content = all_content
        answer_sheet = "\n# Answer for Problem {0}\n#\n".format(args['probnum'])
    # copy assets if any needed
    if len(assets_required)>0:
        toDirectory = os.path.join(dirname,"assets_{}".format(args['probnum']))
        os.makedirs(toDirectory)
        for asset in assets_required:
            shutil.copy(os.path.join(problem['directory'],"assets/"+asset),toDirectory)
    return problem_content, answer_sheet, answer_key

setattr(AssignmentGenerator, 'processOneProblem', processOneProblem)

In [50]:
def processAssignment(self):
    for student in (self.roster if self.roster else [["Instructor","The","0"]]):
        sid, lastname, firstname = student[0], student[1], student[2]
        self.info_for_grading[(sid, lastname, firstname)] = []
        roster_present = (self.roster!=None) and not (student == ["","",""])
        stdcode = None
        if roster_present:
            seed = random.seed(sid)
            stdcode = ""
            student_subdirname = "{ln}_{fn}_{sid:01d}".format(
                ln=lastname.lower(),fn=firstname.lower(),sid=int(sid)%10)
            stdcode += str(int(sid)%10)
            work_dirname = os.path.join(self.output_dirname,student_subdirname)
            sid_trunc = sid
            while os.path.exists(work_dirname):
                sid_trunc = sid_trunc//10
                student_subdirname = student_subdirname + str(int(sid_trunc)%10)
                stdcode += str(int(sid_trunc)%10)
            os.makedirs(work_dirname)
            assignmentfile = os.path.join(work_dirname,"assignment.md")
            answerfile = os.path.join(work_dirname,"answers.txt")
        else:
            seed = random.seed(0)
            work_dirname = self.output_dirname
            assignmentfile = os.path.join(work_dirname,"assignment.md")
            answerfile = os.path.join(work_dirname,"answers.txt")
        with open(assignmentfile,'w') as f1, open(answerfile,'w') as f2:
            if 'title' in self.data:
                print("---\ntitle: {0}".format(self.data['title']),file=f1)
                if 'instructor' in self.data:
                    print("author: {0}".format(self.data['instructor']),file=f1)
                print("header-includes: |",file=f1)
                print(" \\usepackage{fancyhdr}",file=f1)
                print(" \\pagestyle{fancy}",file=f1)
                print(" \\fancyhead[CO,CE]{Do not distribute.}",file=f1)
                if roster_present or self.roster==None:
                    print(" \\fancyfoot[CO,CE]{{Created for {0} {1}}}".format(firstname, lastname),file=f1)
                print(" \\fancyfoot[LE,RO]{\\thepage}",file=f1)
                print("---",file=f1)
                print("## {0}".format(self.data['title']), file=f1)
                print("---", file=f2)
                print("title: '{0}'".format(self.data['title']), file=f2)
            else:
                print("## Assignment", file=f1)
                print("title: Assignment", file=f2)
            if 'course' in self.data:
                print("### {0}".format(self.data['course']), file=f1)
                print("course: '{0}'".format(self.data['course']), file=f2)
            if 'deadline' in self.data:
                print("### Due on {0}".format(self.data['deadline']), file=f1)
                print("deadline: '{0}'".format(self.data['deadline']), file=f2)
            if roster_present:
                print("### Student Name: {0} {1}".format(firstname, lastname), file=f1)
                print("student: {0} {1}".format(firstname, lastname), file=f2)
                if stdcode!=None:
                    print("code: {0}".format(stdcode), file=f2)
                #print("### Student Name: {0} {1} (XXXXX{2})".format(firstname, lastname, sid[-4:]), file=f1)
                #print("student: {0} {1} (XXXXX{2})".format(firstname, lastname, sid[-4:]), file=f2)
            if self.instructions:
                print("### Instructions:", file=f1)
                f1.write(self.instructions+"\n")
            print("instructions:", file=f2)
            print("  - 'Do not edit lines starting with # or ---.'", file=f2)
            print("  - You may add lines for your answer.", file=f2)
            print("  - 'Do not start with you answer lines with # or ---.'", file=f2)
            print("---\n", file=f2)

            probnum = 0
            for p in self.problems:
                probnum += 1
                if 'subproblems' in p or (('randompicks' in p) and self.roster==None):
                    f1.write("### Problem {0}:".format(probnum))
                    #f2.write("---\n# Answer {0}\n#\n".format(probnum))
                    if 'title' in p:
                        f1.write(" {}".format(p['title']))
                    if 'points' in p:
                        f1.write("    [{} points]".format(p['points']))
                    f1.write("\n")
                    subprobnum = 0
                    for sp in (p['subproblems'] if 'subproblems' in p else p['randompicks']):
                        subprobnum += 1
                        if 'subproblems' in p:
                            f1.write("#### Subproblem {0}.{1}:".format(probnum,subprobnum))
                        else:
                            f1.write("### Problem {0} ~[ver{1}]~:".format(probnum,subprobnum))
                        if 'extra_credit' in sp and int(sp['extra_credit'])!=0:
                            f1.write(" Extra Credit:")
                        if 'title' in sp:
                            f1.write(" {}".format(sp['title']))
                        if 'points' in sp:
                            f1.write("    [{} points]".format(sp['points']))
                        f1.write("\n")
                        if 'subproblems' in p:
                            print("---\n# Answer {0}.{1}\n#".format(probnum,subprobnum), file=f2)
                        else:
                            print("---\n# Answer {0} [ver{1}]\n#".format(probnum,subprobnum), file=f2)
                        args = {} #'lastname':lastname, 'firstname':firstname, 'sid':sid}
                        args['probnum'] = "{0}_{1}{2}".format(probnum,("ver" if self.roster==None else ""),subprobnum)
                        args['probpoints'] = str(sp.get('points',0))
                        args['probfile'] = sp.get('file',"")
                        problem_content, answer_sheet, answer_key = self.processOneProblem(sp,args,work_dirname)
                        args['probnum'] = str(probnum)
                        if 'subproblems' in p:
                            args['subprobnum'] = str(subprobnum)
                        else: 
                            args['version'] = str(subprobnum)                            
                        args['answer_key'] = answer_key
                        f1.write(problem_content+"\n")
                        f2.write(answer_sheet+"\n")
                        self.info_for_grading[(sid, lastname, firstname)].append(args)
                else:
                    if 'randompicks' in p:
                        is_randompick = True
                        i = random.randrange(len(p['randompicks']))
                        p = p['randompicks'][i]
                        f1.write("### Problem {0} ~[ver{1}]~:".format(probnum,i+1))
                        f2.write("---\n# Answer {0} [ver{1}]\n#\n".format(probnum,i+1))
                    else:
                        is_randompick = False
                        f1.write("### Problem {0}:".format(probnum))
                        f2.write("---\n# Answer {0}\n#\n".format(probnum))
                    if 'extra_credit' in p and int(p['extra_credit'])!=0:
                        f1.write(" Extra Credit:")
                    if 'title' in p:
                        f1.write(" {}".format(p['title']))
                    if 'points' in p:
                        f1.write("    [{} points]".format(p['points']))
                    f1.write("\n")
                    args = {} #'lastname':lastname, 'firstname':firstname, 'sid':sid}
                    args['probnum'] = str(probnum) if (not is_randompick) else "{0}_ver{1}".format(probnum,i+1)
                    args['probpoints'] = str(p.get('points',0))
                    args['probfile'] = p.get('file',"")
                    problem_content, answer_sheet, answer_key = self.processOneProblem(p,args=args,dirname=work_dirname)
                    args['probnum'] = str(probnum)
                    if (is_randompick): 
                        args['version'] = str(i+1)
                    args['answer_key'] = answer_key
                    f1.write(problem_content+"\n")
                    f2.write(answer_sheet+"\n")
                    self.info_for_grading[(sid, lastname, firstname)].append(args)                            

setattr(AssignmentGenerator, 'processAssignment', processAssignment)

def saveGradingInfo(self):
    with open(os.path.join(self.dirname_dst,"grading_info.pkl"),"wb") as f:
        pickle.dump(self.info_for_grading,f)
    rows = []
setattr(AssignmentGenerator, 'saveGradingInfo', saveGradingInfo)

In [51]:
a = AssignmentGenerator()
a.resetData()
a.loadData()
a.readRosterFile()
a.processAssignment()
a.saveGradingInfo()
print("DONE!")

DONE!


In [52]:
pickle.load(open('/Users/mbs/Courses/ECEM16/Spring 2020/psets/pset1/grading_info.pkl','rb'))

{('605391825',
  'ANDERSEN',
  'DESMOND'): [{'probnum': '1',
   'probpoints': '5',
   'probfile': 'digital_abstraction/color_wheel.md',
   'answer_key': [{'type': 'int', 'answer': '[4, [5, 6]]'},
    {'type': 'other',
     'helper': 'graderColorWheel',
     'format': '^([a-z\\-]+:\\s*[01\\s]*(\\n)?){12}$',
     'answer': ''}]}, {'probnum': '2',
   'probpoints': '5',
   'probfile': 'digital_abstraction/genetic_code_ankur.md',
   'answer_key': [{'type': 'int', 'answer': '2'},
    {'type': 'int', 'answer': '5'},
    {'type': 'int', 'answer': '3'},
    {'type': 'int', 'answer': '3'},
    {'type': 'int', 'answer': '27'},
    {'type': 'int', 'answer': '17'},
    {'type': 'int', 'answer': '12'}]}, {'probnum': '3',
   'probpoints': '3',
   'probfile': 'boolean_algebra/circuit_to_dual_1.md',
   'subprobnum': '1',
   'answer_key': [{'type': 'other',
     'answer': '(a | c | d) & (~a | ~b | ~c) & (~a | ~b | ~d) & (b & c & d)',
     'helper': {'func': 'boolEquivalentExpression',
      'vars': ['a'

In [37]:
def main(args):
    a = AssignmentGenerator(args.infile[0] if args.infile else DEFAULT_INFILE, 
                            args.destdir[0] if args.destdir else DEFAULT_DESTDIR)
    a.resetData()
    a.loadData()
    if not args.instructor:
        a.readRosterFile(args.roster)
    a.processAssignment()
    a.saveGradingInfo()
    print("DONE!")

In [22]:
import argparse

if __name__ == "__main__":
    # execute only if run as a script
    parser = argparse.ArgumentParser(description='Generate course assignment.')
    parser.add_argument("infile",nargs='?',default=None,help="Assignment specification file (YAML).")
    parser.add_argument("-d","--destdir",nargs=1,default=None,help="Destination directory.")
    parser.add_argument("-r","--roster",nargs=1,default=None,help="Roster file.")
    parser.add_argument("-i","--instructor",help="Instructor mode.",action='store_true')
    args = parser.parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [-d DESTDIR] [-r ROSTER] [-i] [infile]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [466]:
# pandoc assignment.md -o assignment.pdf -V geometry:margin=1in 
# pandoc assignment.md -o assignment.html --mathml -s --metadata title=" "

In [372]:
# pickle.load(open('/Users/mbs/Courses/ECEM16/Spring 2020/psets/pset1/grading_info.pkl','rb'))